## The Backstory

I came across the post below while doom-scrolling instagram and wondered how accurate it was - after all, a 100% success rate is rare in anything. After looking into it a bit, more questions started coming up and the urge to answer them didn't die down so I decided I would write down the questions and wait until I got home for the satisfaction of responding these questions myself.

- Did the person who created the spreadsheet nit-pick a one year holding period intentionally?
- Would a 3 month or 4 year holding period result in less (or more) impressive returns?
- Does this strategy truly have a 100% success rate?

<p align="center">
    <img src="./images/original_post.png" alt="Original Post" width="400"/>
</p>


### Extracting Data from Image

Long time ago I tried using OCR to extract live timing data from an F1 feed. The extracted data was not very accurate ,so I scrapped the project. At the time, however, Optical Character Recognition (OCR) was at an earlier stage and there weren't many open source libraries available, but trying OCR once again was the only way of extracting the data from the image so I set off on some research for more recent open source OCR projects. 

Easy, quick google search and 'pytesseract' came up - it was perfect - so I glanced through the documentation and came up with the below script.

In [1]:
import cv2
import pytesseract
import pandas as pd

# Pre-process image for better results
image_path = "images/cropped_table.png" 
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
_, thresh = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Extract text to string
custom_config = r'--oem 3 --psm 6'  # OCR Engine Mode and Page Segmentation Mode
text = pytesseract.image_to_string(thresh, config=custom_config)

# Strip string 
rows = text.strip().split("\n")
df = pd.DataFrame([row.split() for row in rows if row.strip()])

# Save to csv
print(df.head)

<bound method NDFrame.head of             0      1         2        3            4      5        6   \
0       1-Year  Total    1-Year    Total       1-Year     _‘    Total   
1      Markets     in    S&P500  Forward      Returns      |  Markets   
2      Turmoil  Close  __Return    Since  _[inTurmoil  Close   Return   
3     5/6/2016   1128       21%     367%     3/5/2020   3024      29%   
4     5/7/2010   1111       24%     374%     3/6/2020   2972      33%   
5     5/9/2010     nu       24%     374%     3/8/2020   2972      33%   
6     6/4/2010   1065       23%     393%     3/9/2020   2787      44%   
7     8/4/2011   1200       19%     328%    3/10/2020   2882      39%   
8     8/7/2011   1199       19%     328%    3/11/2020   2741      46%   
9     8/8/2011   1119       28%     359%    3/12/2020   2481      63%   
10    8/9/2011   1173       22%     338%    3/13/2020   2711      49%   
11   8/10/2011   1122       28%     358%    3/15/2020   2711      49%   
12   8/11/2011   1173

### Cleaning Raw Data

The data extracted from the image was not perfect, but some cleaning helped out. For the purpose of answering the questions, the degree of certainty obtained from this dataset should be enough.

Additionally, I used the 'yfinance' API to pull the data for the index (^SPX).

In [2]:
# Drop the last three columns (empty) & drop first 3 rows
df = df.iloc[3:, :-3]

# Extract every set of 4 columns and prepare for concatenation
common_columns = ["date", "spx_close", "1_y_return", "return_by_05-05-2022"]
dfs = [df.iloc[:, i:i+4].set_axis(common_columns, axis=1) 
       for i in range(0, df.shape[1], 4)]
df = pd.concat(dfs, ignore_index=True)

# Strip "|", "[", and "(" from dates & replace missing/wrong data with none
df["date"] = df["date"].astype(str).str.replace(r'[|\[(]', '', regex=True).str.strip()
df.replace(["nu", "?", "Th", "a%", "S4A%", "4a%", "4A%"], None, inplace=True)

# Fix data on index 73
df.loc[73, "date"] = df.loc[73, "spx_close"] 
df.loc[73, "spx_close"] = df.loc[73, "1_y_return"]
df.loc[73, "1_y_return"] = None

# Remove row with missing data
df = df[df["date"] != "4147"].reset_index(drop=True)

# Conver date to datetime
df['date'] = pd.to_datetime(df["date"])
image_df = df.sort_values(by="date", ascending=True, ignore_index=True)

# Save
image_df

,date,spx_close,1_y_return,return_by_05-05-2022
0,2010-05-07,1111,24%,374%
1,2010-05-09,None,24%,374%
2,2010-06-04,1065,23%,393%
3,2011-08-04,1200,19%,328%
4,2011-08-07,1199,19%,328%
...,...,...,...,...
90,2020-05-29,3044,40%,40%
91,2020-06-01,3056,40%,40%
92,2020-06-02,3081,38%,38%
93,2020-06-03,3123,38%,37%


In [3]:
import yfinance as yf
from datetime import datetime

# Download historical daily price data
ticker = "^SPX"
start_date = "2010-05-07"
end_date = datetime.today().strftime('%Y-%m-%d')
spx_df = yf.download(ticker, start=start_date, end=end_date, interval="1d")

# Drop a level as data is provided as MultiIndex
spx_df.columns = spx_df.columns.droplevel(1)

print(spx_df.head())

[*********************100%***********************]  1 of 1 completed

Price             Close         High          Low         Open      Volume
Date                                                                      
2010-05-07  1110.880005  1135.130005  1094.150024  1127.040039  9472910000
2010-05-10  1159.729980  1163.849976  1122.270020  1122.270020  6893700000
2010-05-11  1155.790039  1170.479980  1147.709961  1156.390015  5842550000
2010-05-12  1171.670044  1172.869995  1155.430054  1155.430054  5225460000
2010-05-13  1157.439941  1173.569946  1156.140015  1170.040039  4870640000


### Back On Track: Answering the questions.

- ##### Did the person who created the spreadsheet nit-pick a one year holding period intentionally?
- ##### Would a 3 month or 4 year holding period result in less (or more) impressive returns?
- ##### Does this strategy truly have a 100% success rate?

Now, to answer this question I need to:
1. Find the dates 1, 3, 6 month and 1, 2, 3, 4 years from episode date
2. Merge values in close column based on 
3. Find average return by holding period and confirm if all values are positive/negative


In [ ]:
import pandas as pd
from datetime import timedelta

# Keep only necessary columns
spx_df = spx_df[["Close"]].copy()
df = image_df[["date"]].copy()

# Define time shifts
time_shifts = {
    "1m": timedelta(days=30),
    "3m": timedelta(days=90),
    "6m": timedelta(days=180),
    "1y": timedelta(days=365),
    "2y": timedelta(days=730),
    "3y": timedelta(days=1095),
    "4y": timedelta(days=1460),
}

# Create future date columns
for label, delta in time_shifts.items():
    df[f"date_{label}"] = df["date"] + delta

# Merge data
df = pd.merge_asof(
    df.sort_values("date"),
    spx_df.rename(columns={"Close": "original_close"}), 
    left_on="date",
    right_index=True,
    direction="backward"  # Find the closest past date
)

# Perform nearest merge for each future date
for label in time_shifts.keys():
    df = pd.merge_asof(
        df.sort_values(f"date_{label}"),
        spx_df.rename(columns={"Close": f"Close_{label}"}), 
        left_on=f"date_{label}",
        right_index=True,
        direction="backward"
    )

# Calculate percentage difference for each future close price
for label in time_shifts.keys():
    df[f"pct_change_{label}"] = (df[f"Close_{label}"] - df["original_close"]) / df["original_close"] * 100

# Calculate additional metrics for each holding period
metrics = {}
for label in time_shifts.keys():
    metrics[label] = {
        "Average Return (%)": df[f"pct_change_{label}"].mean(),
        "Median Return (%)": df[f"pct_change_{label}"].median(),
        "Min Return (%)": df[f"pct_change_{label}"].min(),
        "Max Return (%)": df[f"pct_change_{label}"].max(),
        "Standard Deviation (%)": df[f"pct_change_{label}"].std(),
        "Success Rate (%)": (df[f"pct_change_{label}"] > 0).mean() * 100,
    }

# Convert metrics dictionary to DataFrame
metrics_df = pd.DataFrame.from_dict(metrics, orient="index")
metrics_df

,Average Return (%),Median Return (%),Min Return (%),Max Return (%),Standard Deviation (%),Success Rate (%)
1m,3.943433,4.528182,-23.259623,25.114427,8.322019,76.842105
3m,12.963049,13.844180,-8.383421,38.452674,8.391308,94.736842
6m,19.761233,19.768343,-2.694721,48.362837,9.767593,98.947368
1y,37.678413,40.806183,3.350775,74.779663,16.078107,100.000000
2y,48.915179,44.758847,20.017282,99.170485,18.248482,100.000000
3y,49.968028,46.243974,24.378705,89.879979,12.264099,100.000000
4y,80.088797,78.745667,32.401021,133.940306,18.360739,100.000000


### BINGO! 

So we weren't being lied to - no nit-picking here. Nice. The original post did indicate 40% average yearly return and our calculations indicate 37.68%, but I'll just attribute the discrepancy to the rounding applied to the price in the original post in addition to a possible discrepancy between the dates selected (merging the dates forward or backwards). Anyways, all of the chosen holding periods yielded positive returns on average. 

In addition to the above, a 100% success rate is accurate not only for the 1 year holding period, but also all other greater holding periods tested.


### Next up: 

Naturally - I was left with more questions than answers. 

- What does the distribution of returns look like for the different periods of this strategy?
- Sure, the S&P might have had these returns. But what about other assets? Microcaps, mid-cap, oil, gold, 10y treasuries, bitcoin? How would different sectors have performed, on average, against each other?
- The S&P has had, historically, positive average returns. Is there a marginal or statistically significant advantage to buying at this particular point compared to a random entry?

But these questions are meant to be answered at another time.